In [100]:
# import relevant packages

from twython import Twython
import pandas as pd
import json
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import nltk 
import string
import re
%matplotlib inline

In [101]:
# hydrate the tweets
# note: this cell only needs to be run if the tweets
# are not in a hydrated state already, hence the code
# below being commented out

#!twarc hydrate hellobrother_idlist.txt > hellobrother_rehydrated.jsonl

In [102]:
# store twitter api info as strings in variables

api_key = 'q6TGpJb1yW6NqdoKuNDNUnlQ1'
api_secret = 'Fun91BImufQJLhPWf0hlDl9XQDnIDZkY97M3EjMeuixiYTWtMT'
access_token = '122907048-g8WJABKyETqczTBVrKVYRkxSSL50AEwIL8M2xCbK'
access_secret = 'KlirgdNmeGxBAEsCrC7GxhCvc7P1URJVvQDUo3RgLB1zF'

In [103]:
# create a Twython variable using twitter api info

twitter = Twython(api_key, api_secret, access_token, access_secret)

# hydrate one tweet for proof of concept

tweet = twitter.show_status(id='795952534565298177', include_entities=True)
tweet

{'created_at': 'Tue Nov 08 11:34:06 +0000 2016',
 'id': 795952534565298177,
 'id_str': '795952534565298177',
 'text': 'todays the day oh #ElectionDay',
 'truncated': False,
 'entities': {'hashtags': [{'text': 'ElectionDay', 'indices': [18, 30]}],
  'symbols': [],
  'user_mentions': [],
  'urls': []},
 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>',
 'in_reply_to_status_id': None,
 'in_reply_to_status_id_str': None,
 'in_reply_to_user_id': None,
 'in_reply_to_user_id_str': None,
 'in_reply_to_screen_name': None,
 'user': {'id': 3000336986,
  'id_str': '3000336986',
  'name': 'eva',
  'screen_name': 'goodheartedjc',
  'location': '',
  'description': '',
  'url': 'https://t.co/VKMLnA4W85',
  'entities': {'url': {'urls': [{'url': 'https://t.co/VKMLnA4W85',
      'expanded_url': 'http://hstyles.co.uk',
      'display_url': 'hstyles.co.uk',
      'indices': [0, 23]}]},
   'description': {'urls': []}},
  'protected': False,
  'followers_count':

In [104]:
# each id has its own line in election-day.txt
# counting lines to check how many tweets have been lost to deletion or other factors

num_lines = sum(1 for line in open('election-day.txt'))
print(num_lines)

3630769


In [105]:
# Read each line of the tweetset, open them as JSON objects,
# and finally create a Pandas DataFrame of these objects

with open('electionday_rehydrated.jsonl') as f:
    tweets_jsonl = f.read()

df_hydrated = pd.DataFrame([json.loads(jline) for jline in
tweets_jsonl.splitlines()])

# check how many tweets were retrieved from the dataset and display
# as a percentage of the total amount of tweets in the dataset

len(df_hydrated.index)/3630769*100

5.194299058959686

In [106]:
# give us an overview of the dataframe

df_hydrated.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188593 entries, 0 to 188592
Data columns (total 34 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   created_at                 188593 non-null  object 
 1   id                         188593 non-null  int64  
 2   id_str                     188593 non-null  object 
 3   full_text                  188593 non-null  object 
 4   truncated                  188593 non-null  bool   
 5   display_text_range         188593 non-null  object 
 6   entities                   188593 non-null  object 
 7   extended_entities          58334 non-null   object 
 8   source                     188593 non-null  object 
 9   in_reply_to_status_id      2233 non-null    float64
 10  in_reply_to_status_id_str  2233 non-null    object 
 11  in_reply_to_user_id        3822 non-null    float64
 12  in_reply_to_user_id_str    3822 non-null    object 
 13  in_reply_to_screen_name    38

In [107]:
# cleaning the data

# drop all columns not listed in features table, see poster
# required: id, full_text, user

df = df_hydrated.loc[:, ['id', 'full_text', 'user']]
print(df.columns.to_list())

# check for null values

df.isnull().values.any()

['id', 'full_text', 'user']


False

In [108]:
# extract required information from user column and drop everything else

df['name'] = df['user'].str['name']
df['screen_name'] = df['user'].str['screen_name']
df['created_at'] = df['user'].str['created_at']
df['followers_count'] = df['user'].str['followers_count']
df['friends_count'] = df['user'].str['friends_count']
df['verified'] = df['user'].str['verified']
df = df.drop(['user'], axis=1)

df.head(3)

,id,full_text,name,screen_name,created_at,followers_count,friends_count,verified
0,795952533516808192,I'm with Purr! Vote Peppah!!! #americafirst #E...,PeppahforPrez2016,timeisMEOW2016,Wed Jul 06 00:19:13 +0000 2016,11,116,False
1,795952534565298177,todays the day oh #ElectionDay,eva,goodheartedjc,Thu Jan 29 02:36:14 +0000 2015,2607,89,False
2,795952536985473024,RT @GreekAnalyst: The rest of the world today....,mery,93smixtape,Wed Jan 02 13:15:54 +0000 2013,1141,197,False


In [109]:
# clean tweet text

# remove punctuation

def remove_punct(text):
    text  = "".join([char for char in text if char not in string.punctuation])
    text = re.sub('[0-9]+', '', text)
    return text

df['text_punc'] = df['full_text'].apply(lambda x: remove_punct(x))
df = df.drop(['full_text'], axis=1)
df.head(3)

,id,name,screen_name,created_at,followers_count,friends_count,verified,text_punc
0,795952533516808192,PeppahforPrez2016,timeisMEOW2016,Wed Jul 06 00:19:13 +0000 2016,11,116,False,Im with Purr Vote Peppah americafirst Election...
1,795952534565298177,eva,goodheartedjc,Thu Jan 29 02:36:14 +0000 2015,2607,89,False,todays the day oh ElectionDay
2,795952536985473024,mery,93smixtape,Wed Jan 02 13:15:54 +0000 2013,1141,197,False,RT GreekAnalyst The rest of the world today El...


In [110]:
# tokenise text

def tokenization(text):
    text = re.split('\W+', text)
    return text

df['text'] = df['text_punc'].apply(lambda x: tokenization(x.lower()))
df = df.drop(['text_punc'], axis=1)
df.head(3)

,id,name,screen_name,created_at,followers_count,friends_count,verified,text
0,795952533516808192,PeppahforPrez2016,timeisMEOW2016,Wed Jul 06 00:19:13 +0000 2016,11,116,False,"[im, with, purr, vote, peppah, americafirst, e..."
1,795952534565298177,eva,goodheartedjc,Thu Jan 29 02:36:14 +0000 2015,2607,89,False,"[todays, the, day, oh, electionday]"
2,795952536985473024,mery,93smixtape,Wed Jan 02 13:15:54 +0000 2013,1141,197,False,"[rt, greekanalyst, the, rest, of, the, world, ..."


In [111]:
# remove stopwords from text

stopword_list = ['rt','the','of','a','and','today','is']

df['text'] = df['text'].apply(lambda x: [i for i in x if i not in stopword_list])

# tidy up created_at column for comparison

df['created_at']=pd.to_datetime(df['created_at']).dt.date
df.head(3)

,id,name,screen_name,created_at,followers_count,friends_count,verified,text
0,795952533516808192,PeppahforPrez2016,timeisMEOW2016,2016-07-06,11,116,False,"[im, with, purr, vote, peppah, americafirst, e..."
1,795952534565298177,eva,goodheartedjc,2015-01-29,2607,89,False,"[todays, day, oh, electionday]"
2,795952536985473024,mery,93smixtape,2013-01-02,1141,197,False,"[greekanalyst, rest, world, electionday, https..."


In [112]:
# create variable for election date

date = 20161008

# convert dates to integers for comparison

df['created_at'] = pd.to_datetime(df['created_at'])
df['created_at'] = df['created_at'].dt.strftime("%Y%m%d").astype(int)

In [113]:
# create new column with label for created_at for all accounts created 1 month 
# before the election day. New accounts are likely to be bots. (TRUE = CREATED OVER 1 MONTH BEFORE ELECTION, 1)

df['created_at_label'] = df['created_at'] < date

# create new column with label for name for all accounts containing numbers in name (TRUE = CONTAINS NUMBERS, 1)

df['name_label'] = df['name'].str.contains("0|1|2|3|4|5|6|7|8|9")

# create new column for friends to followers count (TRUE = MORE FOLLOWING, 1)

df['diff_label'] = df['followers_count'] < df['friends_count']

# strip text column down to just words for encoding

df['text'] = df['text'].agg(lambda x: ','.join(map(str, x)))
df['text'] = df['text'].str.replace(',',' ')

# convert boolean columns to integers for use in machine learning

df["name_label"] = df["name_label"].astype(int)
df["diff_label"] = df["diff_label"].astype(int)
df["created_at_label"] = df["created_at_label"].astype(int)
df["verified"] = df["verified"].astype(int)

df

,id,name,screen_name,created_at,followers_count,friends_count,verified,text,created_at_label,name_label,diff_label
0,795952533516808192,PeppahforPrez2016,timeisMEOW2016,20160706,11,116,0,im with purr vote peppah americafirst election...,1,1,1
1,795952534565298177,eva,goodheartedjc,20150129,2607,89,0,todays day oh electionday,1,0,0
2,795952536985473024,mery,93smixtape,20130102,1141,197,0,greekanalyst rest world electionday httpstcoej...,1,0,0
3,795952538633768960,Charles McCool ✈️,CharlesMcCool,20081221,28577,6458,1,mrscotteddy all places you can get free food f...,1,0,0
4,795952539086848000,Anker Vladwin 32,Ankvladwin,20130508,1721,470,0,esta vieja usa mejor el castellano que algunos...,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...
188588,796002359621128192,The N,ImLilMissPlaced,20100506,681,1157,0,i made playlist for electionday because you kn...,1,0,1
188589,796002359675592704,Madeline Ashby,MadelineAshby,20090419,6088,2502,0,clintsmithiii never forget group black teenage...,1,0,0
188590,796002359721807872,joe maragh,joemaragh133,20161108,5,17,0,azymanzur for justin bieber like for donald tr...,0,0,1
188591,796002360598335488,ruth navarrete,ruthnavarrete98,20150309,93,667,0,azymanzur for justin bieber like for donald tr...,1,0,1


In [114]:
# encode text column

from sklearn.neighbors import KNeighborsClassifier
from sklearn import preprocessing
from sklearn import metrics
import math

# creating labelEncoder
le = preprocessing.LabelEncoder()

# encode text
df['text_label'] = le.fit_transform(df['text'])

# add column for real or not to use as label to test for

conditions = [
    (df['verified'] == 0) & (df['created_at_label'] == 0) & (df['name_label'] == 1) & (df['diff_label'] == 1),
    (df['verified'] == 0) & (df['created_at_label'] == 0) & (df['name_label'] == 0) & (df['diff_label'] == 1)]
choices = [1 , 1]

df['bot_or_not'] = np.select(conditions, choices, default=0)

# get an idea of the distribution of bots

df.bot_or_not.value_counts()

0    185784
1      2809
Name: bot_or_not, dtype: int64

In [115]:
# drop non-numeric columns

df = df.drop(['name','screen_name','text'], axis=1)

In [116]:
# split datasets into training and testing

x = df.iloc[:, :-1]
y = df.iloc[:, -1]

from sklearn import metrics
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)

In [129]:
# KNN

print('KNN Classifier Algorithm')

# create KNN Classifier
knn = KNeighborsClassifier(n_neighbors=10)

# train the model using the training sets
knn.fit(x_train, y_train)

# predict the response for test dataset
pred1 = knn.predict(x_test)
print(f"Predicted: {pred1}")

 
# RMSE Computation
rmse = np.sqrt(MSE(y_test, pred1))
print("RMSE : % f" %(rmse))

# model accuracy, how often is the first classifier correct?

print(f"Classification Report:\n {metrics.classification_report(y_test, pred1)}")
print(f"Confusion Matrix:\n {metrics.confusion_matrix(y_test, pred1)}")

# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, pred1))

KNN Classifier Algorithm
Predicted: [0 0 0 ... 0 0 0]
RMSE :  0.119318
Classification Report:
               precision    recall  f1-score   support

           0       0.99      1.00      0.99     37175
           1       0.73      0.02      0.04       544

    accuracy                           0.99     37719
   macro avg       0.86      0.51      0.52     37719
weighted avg       0.98      0.99      0.98     37719

Confusion Matrix:
 [[37171     4]
 [  533    11]]
Accuracy: 0.9857631432434582


In [132]:
# XGBOOST
# import needed packages

print('XGBoost Algorithm')

import xgboost as xg
from sklearn.metrics import mean_squared_error as MSE

# Instantiation
xgb_r = xg.XGBClassifier()
 
# Fitting the model
xgb_r.fit(x_train, y_train)
 
# Predict the model
pred2 = xgb_r.predict(x_test)
print(f"Predicted: {pred2}\n")
 
# RMSE Computation
rmse = np.sqrt(MSE(y_test, pred2))
print("RMSE : % f" %(rmse))

print(f"Classification Report:\n {metrics.classification_report(y_test, pred2)}")
print(f"Confusion Matrix:\n {metrics.confusion_matrix(y_test, pred2)}")

# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, pred2))

XGBoost Algorithm
Predicted: [0 0 0 ... 0 0 0]

RMSE :  0.005149
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     37175
           1       1.00      1.00      1.00       544

    accuracy                           1.00     37719
   macro avg       1.00      1.00      1.00     37719
weighted avg       1.00      1.00      1.00     37719

Confusion Matrix:
 [[37174     1]
 [    0   544]]
Accuracy: 0.9999734881624646


In [127]:
# DECISION TREE

print('Decision Tree Algorithm')

from sklearn.tree import DecisionTreeClassifier

# Instantiation
clf = DecisionTreeClassifier(max_depth =3, random_state = 0)

# fitting the model
clf.fit(x_train, y_train)

# predict the model

pred3 = clf.predict(x_test)
print(f"Predicted: {pred3}\n")

 
# RMSE Computation
rmse = np.sqrt(MSE(y_test, pred3))
print("RMSE : % f" %(rmse))

print(f"Classification Report:\n {metrics.classification_report(y_test, pred3)}")
print(f"Confusion Matrix:\n {metrics.confusion_matrix(y_test, pred3)}")

# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, pred3))

Decision Tree Algorithm
Predicted: [0 0 0 ... 0 0 0]

RMSE :  0.000000
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     37175
           1       1.00      1.00      1.00       544

    accuracy                           1.00     37719
   macro avg       1.00      1.00      1.00     37719
weighted avg       1.00      1.00      1.00     37719

Confusion Matrix:
 [[37175     0]
 [    0   544]]
Accuracy: 1.0


In [128]:
# Random Forest

print('Random Forest Algorithm')

from sklearn.ensemble import RandomForestClassifier

# Instantiation
clf2=RandomForestClassifier(n_estimators=100, random_state=0)

# fitting the model
clf2.fit(x_train, y_train)

# predict the model

pred4 = clf2.predict(x_test)
print(f"Predicted: {pred4}\n")
 
# RMSE Computation
rmse = np.sqrt(MSE(y_test, pred4))
print("RMSE : % f" %(rmse))

print(f"Classification Report:\n {metrics.classification_report(y_test, pred4)}")
print(f"Confusion Matrix:\n {metrics.confusion_matrix(y_test, pred4)}")

# Model Accuracy, how often is the classifier correct?
print("Accuracy: ",metrics.accuracy_score(y_test, pred4))

Random Forest Algorithm
Predicted: [0 0 0 ... 0 0 0]

RMSE :  0.005149
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     37175
           1       1.00      1.00      1.00       544

    accuracy                           1.00     37719
   macro avg       1.00      1.00      1.00     37719
weighted avg       1.00      1.00      1.00     37719

Confusion Matrix:
 [[37174     1]
 [    0   544]]
Accuracy:  0.9999734881624646
